#### Squared differences estimation

Out is the output of our model and it returns a set of 6 arrays representing the curve of infection for the age groups.

Time, t, has its unit step representing a fortnight, thus we will select only the odd entries of the array and compute the first finite differences for each age class to get the number of new cases observed monthly. 

In [23]:
import model as m 
import numpy as np
import matplotlib.pyplot as plt

import pandas as pd

In [24]:
age_class = ["<1", "1-4", "5-9", "10-14", "15-19", "20-29", "30+"]

#### Vector $\beta$ (= force of infection) to estimate

In [3]:
#beta_range = np.arange(3, 15.2, step = 0.2)

In [25]:
#Select disease
d = "Measles"

#monthly data
data = pd.read_csv( d +'_data_processed.csv')
monthly_data = data.iloc[:,1:8]
monthly_data = monthly_data[age_class]

time_date = data["Time"][0:(data.shape[0]-12)]

###### Based on the monthly / cumulative data

In [26]:
new_ic = np.hstack([0.95*(1./7)*np.ones(2), np.zeros(5), 0.05*(1./7)*np.ones(2), np.zeros(5), np.zeros(7), V_ic])

NameError: name 'V_ic' is not defined

In [27]:
out, t = mdl.run(Initial_cond ,t_max = 19,method="RK45", eval_per_year=12)

NameError: name 'mdl' is not defined

In [ ]:
def f_ssq(beta_vec, Initial_cond):
    beta = np.eye(7)*beta_vec
    mdl = m.SIRVModel(para, lambda t: beta)
    out, t = mdl.run(Initial_cond ,t_max = 19,method="RK45", eval_per_year=12)
    
    #Something to do with this monthly cases
    
    monthly_OUT = out[7:(7*2),:]
    groups = monthly_OUT.shape[0]
    ssq_diff = 0
    for ii in range(groups):
    Inf_group = OUT[ii,:]
    cum_infected_sim = Inf_group[1::2]
    #monthly data
    monthly_infected_sim = np.diff(cum_infected_sim)
    #adding the squared diff
    ssq_diff += (monthly_infected_sim - monthly_data[age_class[ii]])**2

In [ ]:
monthly_OUT = out[7:(7*2),1::2]

groups = monthly_OUT.shape[0]
ssq_diff = 0
for ii in range(groups):
    Inf_group = OUT[ii,:]
    cum_infected_sim = Inf_group[1::2]
    #monthly data
    monthly_infected_sim = np.diff(cum_infected_sim)
    #adding the squared diff
    ssq_diff += (monthly_infected_sim - monthly_data[age_class[ii]])**2

In [ ]:
fit_measles = scipy.optimize.fmin(f_ssq, x0 = beta = [0.98, 0.9, 0.85, 0.8, 0.7, 0.7, 0.7] , args=(new_ic))